In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#载入数据集
mnist=input_data.read_data_sets("MNIST_data",one_hot=True)    #我们要将mnist的0-9的标签转化为“one-hot vectors”。具体见课件

#每个批次的大小100
batch_size=100                      #以矩阵形式放入100张照片进行训练
n_batch=mnist.train.num_examples//batch_size     #数据集训练数量整除批次大小，就是一共需要训练多少个批次

#定义两个placeholder
x=tf.placeholder(tf.float32,[None,784])                  #28*28=784
y=tf.placeholder(tf.float32,[None,10])
keep_prob=tf.placeholder(tf.float32)

#创建一个简单的神经网络
W1=tf.Variable(tf.truncated_normal([784,2000],stddev=0.1))     #用truncated_normal函数初始化权值
#truncated_normal:从截断的正态分布中输出随机值。生成的值服从具有指定平均值和标准偏差的正态分布，如果生成的值大于平均值2个标准偏差的值则丢弃重新选择。
b1=tf.Variable(tf.zeros([2000])+0.1)
L1=tf.nn.tanh(tf.matmul(x,W1)+b1)
L1_drop=tf.nn.dropout(L1,keep_prob)         #dropout:设置百分之多少的神经元是工作的，keep_prob值为1是百分之百，值为0.5是百分之50。

W2=tf.Variable(tf.truncated_normal([2000,2000],stddev=0.1))
b2=tf.Variable(tf.zeros([2000])+0.1)
L2=tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)
L2_drop=tf.nn.dropout(L2,keep_prob) 

W3=tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1))
b3=tf.Variable(tf.zeros([1000])+0.1)
L3=tf.nn.tanh(tf.matmul(L2_drop,W3)+b3)
L3_drop=tf.nn.dropout(L3,keep_prob) 

W4=tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b4=tf.Variable(tf.zeros([10])+0.1)
prediction=tf.nn.softmax(tf.matmul(L3_drop,W4)+b4)

#定义二次代价函数
# loss=tf.reduce_mean(tf.square(y-prediction))
#跟softmax搭配使用的交叉熵代价函数
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用梯度下降法
train_step=tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init=tf.global_variables_initializer()

#结果存放在一个布尔型列表中
#argmax返回一维张量中最大的值所在的位置
#tf.argmax(prediction,1) 求prediction最大的位置，并返回这个位置的标签，这里1是数字1。本例中判定概率最大的图片属于哪个标签，并返回标签
#argmax(y,l) 本例中y的值为0或1，哪个位置是1就会返回这个位置。(0 0 1 0) 1在第二个标签位置上，就会返回2.
#本例比较判断出来(predictino)的标签，与真实的标签是否一样，一样的话，就返回True,并存放在correct_prediction里，否则返回False
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) #tf.equal()比较两个变量是否相同，相同的返回True，不同返回False

#求准确率
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))    #cast()函数是转换类型，这里将布尔型转换为浮点型。true--1.0; false--0.0

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(31):                                     #将下面的循环训练31次，就是将所有的图片训练31次
        for batch in range(n_batch):                            #该循环将所有批次的图片拿来训练一次，按批次循环，每批次100张图片
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)#获得一个批次数量的图片（100个），图片数据保存在batch_xs，图片标签存在batch_ys
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
        test_acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})            #喂进去测试集的图片和标签
        train_acc=sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})         #用训练的数据再次进行测试
        print('Iter'+str(epoch)+',Testing Accuracy'+str(test_acc)+',Training Accuracy'+str(train_acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Iter0,Testing Accuracy0.9455,Training Accuracy0.956255
Iter1,Testing Accuracy0.959,Training Accuracy0.972945
Iter2,Testing Accuracy0.9629,Training Accuracy0.981164
Iter3,Testing Accuracy0.9657,Training Accuracy0.985182
Iter4,Testing Accuracy0.9661,Training Accuracy0.987618
Iter5,Testing Accuracy0.9678,Training Accuracy0.989127
Iter6,Testing Accuracy0.9687,Training Accuracy0.990218
Iter7,Testing Accuracy0.97,Training Accuracy0.991036
Iter8,Testing Accuracy0.97,Training Accuracy0.991855
Iter9,Testing Accuracy0.9695,Training Accuracy0.992364
Iter10,Testing Accuracy0.9703,Training Accuracy0.9928
Iter11,Testing Accuracy0.9718,Training Accuracy0.993236
Iter12,Testing Accuracy0.9713,Training Accuracy0.993491
Iter13,Testing Accuracy0.9712,Training Accuracy0.993764
Iter14,Testing Accuracy0.9714,Training 